In [1]:
from CompositeBeam import CompositeBeams

![compositebeamexample2.png](\Resources\Example12_8.png)
![compositebeamexample2-1.png](\Resources\compositebeamexample2-1.png)

In [2]:
cp = CompositeBeams(DeadLoad   = 2.5, #kN/m^2
                    LiveLoad_c = 1   , #kN/m^2

                    # Kompozit çalışma aşaması yükleri
                    SuperDeadLoad = 2.0, #kN/m^2
                    LiveLoad      = 5.0, #kN/m^2
                    w_girder      = 1.0, #kN/m kiriş ağırlığı

                    # Beton bilgisi
                    f_ck     = 30  ,   #N/mm2
                    wc       = 2400,   #kg/m3

                    # Çelik malzeme bilgisi
                    Fy       = 355, #N/mm2
                    Fu_beam  = 510, #N/mm2

                    # Çelik enkesit özellikleri
                    Abeam     = 9880,     #mm^2
                    Hbeam     = 450 ,       #mm
                    Hweb      = 378.8  ,    #mm
                    b_flange  = 190  ,      #mm
                    t_flange  = 14.6   ,     #mm
                    t_web     = 9.4 ,      #mm
                    Ix        = 33_740*10**4, #mm^4
                    Ycon      = 130  ,     #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe

                    # Kiriş uzunluk ve aralık bilgisi
                    Laralık  = 7000,  #mm
                    Lbeam    = 9000,  #mm

                    # Stud çivisi bilgisi
                    D_stud               = 19       , #mm
                    H_stud               = 100      ,  #mm
                    b_studhead           = 32      ,   #mm
                    t_studhead           = 10      ,  #mm
                    Fy_stud              = 235     ,   #N/mm2 çelik ankrajın minimum(karakteristik) akma dayanımı
                    Fu_stud              = 450 ,   #N/mm2 çelik ankrajın minimum(karakteristik) çekme dayanımı
                    CompRatio            = 1,    #Kompozitlik oranı minimum %25 olabilir. %100 tam etkileşimli kompozit demektir.
                    Nw                   = 1,       #Adet Kesit başlığının genişliğinde atılan stud çivisi adeti
                    x                    = 30 ,     #mm Kesit başlığının genişliğinde atılan stud çivileri arasındaki mesafe
                    IsWebAlignmentWelded = False ,  #Stud çivisi kiriş gövdesi izasında mı kaynaklanıyor.

                    # Metal sac bilgisi
                    hr           = 50,    #mm
                    wr           = 165,   #mm
                    t_sac        = 1.2,   #mm
                    RibsDistance = 2*165,   #mm 2*wr
                    IsParallel   = True
    )

In [3]:
print("Design Code Requirements")
print("=="*50)
GeomCheck = cp.ConcAvailableStressCheck(f_ck = cp.f_ck)
GeomCheck = cp.SteelAvailableStressCheck(Fy = cp.Fy)
GeomCheck = cp.MetalDeckCheck(hr = cp.hr, wr = cp.wr)
GeomCheck = cp.ConcreteCoverAboveTopOfHeadedStudAnchorsChecks(hr = cp.hr, t_sac = cp.t_sac, h_stud = cp.H_stud, Ycon =cp.Ycon)
GeomCheck = cp.StudCheck(Ds = cp.D_stud, tf = cp.t_flange, Hs = cp.H_stud, IsWebAlignmentWelded=False)
if GeomCheck != True:
    print("Geometrik kontroller sağlanmalı hesap yapılmadı!!!\n")
# Kiriş enkesit kontrolü
SectionCheck = cp.SectionCheck(Hbeam=cp.Hbeam, t_web=cp.t_web, Fy=cp.Fy)
if SectionCheck != True:
    print("Kesit değiştirin!!!\n")




Design Code Requirements
20 N/mm^2 ≤ 30 N/mm^2 < 70 N/mm^2 √ - TSSDC 12.2.3(a)

355 N/mm^2 ≤ 460 N/mm^2 √ - TSSDC 12.2.3(c)

hr = 50mm ≤ 75mm √ - TSSDC 12.4.2.3

wr = 165mm ≥ 50mm √ - TSSDC 12.4.2.3

h_stud-hr-t_sac = 48.8mm ≥ 38mm √ - TSSDC 12.4.2.3

Ycon - h_stud-t_sac = 28.8mm ≥ 13mm √ - TSSDC 12.4.2.3

Ycon - hr = 80mm ≥ 50mm √ - TSSDC 12.4.2.3

Ds = 19mm ≤ 2.5*tf = 36.5mm √ - TSSDC 12.8.1

Ds = 19mm ≤ 19mm √ - TSSDC 12.8.1

Hs = 100mm ≥ 4*Ds = 76mm √ - TSSDC 12.8.2

h/tw = 47.87 ≤ 3.76*(E/Fy)^0.5 = 89.25 √



In [19]:
print("Yapım aşaması\n")
print("=="*50)
W_G = cp.Calc_Load(Load = cp.DeadLoad, GirdersSpaceLength = cp.Laralık/10**3) + cp.w_girder
W_Q = cp.Calc_Load(Load = cp.LiveLoad_c, GirdersSpaceLength = cp.Laralık/10**3)

print("Yapım aşaması sabit yüklerinden(döşeme ve çelik kiriş) kaynaklı deplasman\n")
delta_cdl = cp.SimpleCompositeBeamDeflection(w=W_G, Lbeam = cp.Lbeam, I = cp.Ix)
print(f"Yapım aşaması sabit yüklerden kaynaklı deplasman (5*W_G*L^4) / (384*E*I) = {delta_cdl}mm")
cp.DeflectionChecks2(delta=delta_cdl, L=cp.Lbeam, Ratio=360)

print("Ters sehim miktarı\n")
delta_c = cp.Camber(delta_cdl = delta_cdl, Lbeam = cp.Lbeam)
delta_yll = cp.SimpleCompositeBeamDeflection(w=W_Q, Lbeam = cp.Lbeam, I = cp.Ix)
print(f"Yapım aşaması hareketli yüklerinden kaynaklı deplasman {delta_yll}mm\n")

delta_ytl = cp.calc_Delta_TL(delta_cdl=delta_cdl, delta_sdl=0., delta_ll=delta_yll,delta_c=delta_c)
print(f"Yapım aşaması toplam düşey yerdeğiştirme = delta_cdl - delta_c + delta_sdl + delta_ll = {delta_ytl}mm, super dead düşey yerdeğiştirmesi yapım öncesi olmadığı için 0.0\n")
cp.DeflectionChecks2(delta=delta_ytl, L=cp.Lbeam, Ratio=240)

Yapım aşaması

Yapım aşaması sabit yüklerinden(döşeme ve çelik kiriş) kaynaklı deplasman

Yapım aşaması sabit yüklerden kaynaklı deplasman (5*W_G*L^4) / (384*E*I) = 23.42mm
23.42 ≤ L/360 = 25.0  √

Ters sehim miktarı

delta_cdl = 23.42mm > 19mm and Lbeam = 9000 > 7600mm olduğu için 17.565mm kadar ters sehime ihtiyaç vardır.

Yapım aşaması hareketli yüklerinden kaynaklı deplasman 8.86mm

Yapım aşaması toplam düşey yerdeğiştirme = delta_cdl - delta_c + delta_sdl + delta_ll = 14.71mm, super dead düşey yerdeğiştirmesi yapım öncesi olmadığı için 0.0

14.71 ≤ L/240 = 37.5  √



True

In [20]:
b_eff       = cp.EffectiveSlabWidth(L = cp.Lbeam, Lu = cp.Laralık) # Effektif döşeme genişliği
tc          = cp.Calc_tc(hr = cp.hr,Ycon = cp.Ycon, IsParallel = cp.IsParallel) # Stud çivisiz beton yüksekliği
Ac          = cp.Calc_Ac(b_eff=b_eff, tc=tc) # Stud çivisiz beton alanı
Cc          = cp.CrushConcCapacity(fck = cp.f_ck,Ac=Ac)
Cs          = cp.YieldBeamCapacity(fy = cp.Fy, As = cp.Abeam)
Cf          = cp.Calc_Cf(Vbeam=Cs, Vcon=Cc)

b_eff = 2250.0mm

tc = 105.0mm
Ac = 236250.0mm^2


In [21]:
# Örnekler kitabında paralel yerleşimden kaynaklı tüm döşeme yüksekliğini almıştır. Burada döşeme yüksekliğinden hadvenin yarı yüksekliği çıkarılmış hali alınmalıydı. Bu nedenle kuvvet kolu mesafesinden tasarım kapasitesi farklı çıkmaktadır.
a           = cp.Calc_a(Cf =Cf, fck = cp.f_ck, b_eff = b_eff)
Y2          = cp.Calc_Y2(Ycon = cp.Ycon, a = a)
Y1          = cp.Calc_Y1(T_steel=Cs, C_conc=Cc, b_flange=cp.b_flange, t_flange = cp.t_flange, t_web = cp.t_web,BeamFy = cp.Fy)
C_flange    = cp.Calc_Cflange(t_flange = cp.t_flange, Y1 = Y1, b_flange = cp.b_flange, BeamFy = cp.Fy)
C_web       = cp.Calc_Cweb(Y1=Y1, t_flange = cp.t_flange, t_web = cp.t_web, BeamFy = cp.Fy)
Mn_web      = cp.PTEInWebMn(T_steel=Cs, C_conc=Cc, Cflange=C_flange, Cweb=C_web, Hbeam = cp.Hbeam, t_flange =cp.t_flange, Y1=Y1, Y2=Y2)
Mn_flange   = cp.PTEInFlangeMn(C_conc=Cc, Cflange=C_flange, Hbeam = cp.Hbeam, BeamAs = cp.Abeam, BeamFy = cp.Fy, Y1=Y1, Y2=Y2)
Mn_slab     = cp.PTEInSlabMn(Cf=Cf, Hbeam = cp.Hbeam, hr = cp.hr, tc=tc, a=a)
Mn_design   = cp.CompositeBeamDesignFlexuralCapacity2(C_conc=Cc, T_steel=Cs,Mn_web=Mn_web, Mn_slab=Mn_slab,Mn_flange=Mn_flange, Y1=Y1, t_flange = cp.t_flange, fi_b=0.9)
CapacityCheck   = cp.CompositeBeamFlexuralCapacityCheck(M_demand=869*10**6, Mn_design=Mn_design)

a = 61.13mm

Y2 = 99.435mm

PTE başlıkta kabul edilirse Y1 = (T_steel - C_conc) / (2*BeamFy*b_flange) = -18.66mm
PTE beton döşeme içerisindedir Y1=-18.66 < 0 ==> Y1 = 0

Mn_web = (C_conc*(Y1+Y2) + 2*Cflange*(Y1 - 0.5*t_flange) + 2*Cweb*(0.5*(Y1-t_flange)) + T_steel*(0.5*Hbeam-Y1)) = [599033728.125 + -0.0 + 711314.92 + 789165000.0] = 1388910043.045 
Mn_flange = (C_conc*(Y1+Y2)) + (2*Csflange*Y1/2) + (BeamAs*BeamFy*( (Hbeam/2) - Y1)) = 599033728.12 +  0.0 + 789165000.0 = 1388198728.125
Mn_slab = Cf * y = 3507400 * 349.44 = 1225625856.0
Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.

φMn = φ * Mn_slab = 0.9 * 1225.626kNm = 1103.063kNm

Talep/Kapasite oranı = 0.79 ≤ 1 √



![Example12_8_answer_c.png](\Resources\Example12_8_answer_c.png)

In [22]:
import math

Ec   = cp.ConcreteYoungModules(fck = cp.f_ck, wc=cp.wc)
A_sa = 3.14 * cp.D_stud**2 /4
Rp   = cp.GetRp(Hstud = cp.H_stud, hr = cp.hr, t_studhead = cp.t_studhead, IsPitchParaleltoBeam = cp.IsParallel)
Rg   = cp.GetRg(StudsNumberInFlangeWidth=cp.Nw, wr = cp.wr, hr = cp.hr, IsPitchParaleltoBeam = cp.IsParallel)
Qn   = cp.OneStudShearCapacity(Asa=A_sa, fck = cp.f_ck, Ec=Ec, Rg=Rg, Rp=Rp, Fu = cp.Fu_stud)
N_ankraj = cp.Calc_StudsNumber(Cf=Cf, CompRatio=cp.CompRatio, Qn=Qn, Nw=cp.Nw, RibsDistance=cp.RibsDistance, L_beam=cp.Lbeam, IsParallel=True)
N_ankraj

Ec = 0.043 * wc **1.5 * fck**0.5 = 0.043 * 2400 **1.5 * 30**0.5 = 27691.465833357393
Rp = 0.75

Rg = 1.0

Qn = 0.5 * Asa * (fck*Ec)**0.5 = 129.145852kN

Qn = 129.145852kN > Rg*Rp*Asa*Fu = 1.0 * 0.75 * 283.385 * 450 = 95.642438kN ==> Qn = 95.642438kN

%100 kompozitlik oranı için belirlenen stud çivisi sayısı 37.



37

In [23]:
Vstud = N_ankraj * Qn
n    = cp.RatioYoungModules(Ec=Ec)
print(f"n = {n}")
Act  = cp.ChengedConcToSteelArea(n=n, Ac=Ac)
print(f"Act = {Act}")
Itr  = cp.Calc_I_tr(Act=Act, n=n, b_eff=b_eff, hr = cp.hr, tc=tc, hb = cp.Hbeam, Ab = cp.Abeam, Ibeam = cp.Ix) #Tametkileşimli kompozit kiriş sehim kontrolü için kullanılır
Ieff = cp.Calc_I_eff(Ibeam = cp.Ix, TotalQn=Vstud, Cf=Cf, Itr=Itr)
I_real = cp.Calc_I_real(I_eff=Ieff) #Kısmi etkileşimli kompozit kiriş sehim kontrolleri için kullanılır.

n = 7.222
Act = 32712.545
Ict = 30054650.720022153
Y =128.4686935824098
Itr = 118133.4498895419cm^4
Ieff = Ibeam + (TotalQn / Cf)**0.5 * (Itr - Ibeam) = 33740.0cm^4 + 1.009 * 84393.4499cm^4 = 118510.0175cm^4


In [34]:
print(f"Yapım öncesi sabit yüklerden kaynaklı sehim (Ters sehim verilmemiştir.) = {delta_cdl}mm")
delta_cll = cp.MainCompositeBeamDeflection(P=52.5*2*10**3 , Lbeam=cp.Lbeam, I=I_real)
print(f"Kompozit kirişte hareketli yükten kaynaklı sehim {round(delta_cll,2)}mm")

delta_sdl = cp.MainCompositeBeamDeflection(P=21*2*10**3 , Lbeam=cp.Lbeam, I=I_real)
print(f"Kompozit kirişte sabit yüklerden kaynaklı sehim {round(delta_sdl,2)}mm")

delta_ctl = cp.calc_Delta_TL(delta_cdl=delta_cdl, delta_sdl=delta_sdl, delta_ll=delta_cll, delta_c=0)
print(f"Kompozit kirişte toplam yük etkisiyle oluşan sehim {round(delta_ctl,2)}mm\n")

cp.DeflectionChecks2(delta=delta_cll,L=cp.Lbeam, Ratio=360)
cp.DeflectionChecks2(delta=delta_ctl,L=cp.Lbeam, Ratio=300)

Yapım öncesi sabit yüklerden kaynaklı sehim (Ters sehim verilmemiştir.) = 23.42mm
Kompozit kirişte hareketli yükten kaynaklı sehim 15.38mm
Kompozit kirişte sabit yüklerden kaynaklı sehim 6.15mm
Kompozit kirişte toplam yük etkisiyle oluşan sehim 44.95mm

15.378446803650794 ≤ L/360 = 25.0  √

44.95 > L/300 = 30.0 X



False

In [26]:
# Ec = ConcreteYoungModules(fck=fck)
# n = RatioYoungModules(Ec=Ec)
# Rp = GetRp(IsPitchParaleltoBeam=False)
# Rg = GetRg(StudsNumberInFlangeWidth=1,wr=wr,hr=hr,IsPitchParaleltoBeam=False)
# Qn = OneStudShearCapacity(Asa=Asa, fck=fck, Ec=Ec, Rg=Rg, Rp=Rp, Fu=Fu)
# TotalQn = ShearStudCapacity(N=N, Qn=Qn)
# Itr = Calc_I_tr(n=n, b_eff=beff,hr=hr,hc=hc,hb=Hbeam,Ab=BeamAs,Ibeam=Ibeam)
# Cf = Calc_Cf(Vbeam=T_steel,Vcon=C_conc)
# Ieff = Calc_I_eff(Ibeam=Ibeam, TotalQn=TotalQn, Cf=Cf, Itr=Itr)